In [1]:
import pandas as pd
import numpy as np
from string import punctuation
import re
import time

from nltk.tokenize import sent_tokenize, word_tokenize, regexp_tokenize # funtions for tokenization
from nltk.tokenize import TweetTokenizer # Class for tweet tokenization
from nltk import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk import download
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

pd.options.display.max_columns=999
pd.options.display.max_colwidth=999

In [2]:
train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')
target = train['target']
test_ids = test['id'].copy()
dframes = [train, test]

In [3]:
# functions for preprocessing

# remove apostrophe from punctuation string

#punctuation = punctuation.replace("'", '') 

def drop_keyword_and_location_cols(df):
    df.drop(['keyword', 'location'], inplace=True, axis='columns')

def avg_word_length(element):
    word_lengths = [len(word) for word in element]
    return np.mean(word_lengths)

def tokenize_words(element):
    tt = TweetTokenizer()
    return tt.tokenize(element.lower())

def remove_punctuation_from_string(element):
    no_punc = element.translate(str.maketrans('', '', punctuation))
    no_punc = no_punc.lower()
    return no_punc

def punctuation_count(element):
    # Not including apostrophes
    counter = 0
    for letter in element:
        if letter in punctuation:
            counter += 1
    return counter

def extract_mentions(element):
    mentions_regex = r"@\w+"
    return regexp_tokenize(element, mentions_regex)

def includes_mention(element):
    if element == 0:
        return 0
    else:
        return 1
    
def extract_stopwords(element):
    try:
        stop_words = stopwords.words('english')
    except LookupError as le:
        download('stopwords')
        stop_words = stopwords.words('english')
    stop_words_present = []
    for word in element:
        if word in stop_words:
            stop_words_present.append(word)
    return stop_words_present

def remove_stopwords(element):
    try:
        stop_words = stopwords.words('english')
    except LookupError as le:
        download('stopwords')
        stop_words = stopwords.words('english')
        
    non_stopwords = []
    for word in element:
        if word not in stop_words:# and word != "'": # clean up stray apostrophes
            non_stopwords.append(word)
    return non_stopwords

def count_numbers(element):
    numbers_regex = r"(\d+\.?,?\s?\d+)"
    return len(regexp_tokenize(element, numbers_regex))

def extract_hashtags(element):
    hashtags_regex = r"#\w+"
    return regexp_tokenize(element, hashtags_regex)

def includes_hashtag(element):
    if element == 0:
        return 0
    else:
        return 1
    
def extract_weblinks(element):
    weblinks = []
    for token in tokenize_words(element):
        if token.startswith('http://'):
            weblinks.append(token)
    return weblinks

def includes_weblink(element):
    if element == 0:
        return 0
    else:
        return 1
    
def remove_links_and_stopwords(element):
    no_links_no_stopwords = []
    stop_words = stopwords.words('english')
    #for word in element.split(' '):
    for word in tokenize_words(element):
        if (word not in stop_words) & (not word.startswith('http')):
            no_links_no_stopwords.append(word)
    return no_links_no_stopwords

def lemmatizing(element):
    lem = WordNetLemmatizer()
    lemmed_words = [lem.lemmatize(word) for word in element]
    return lemmed_words

def remove_numbers(element):
    no_numbers = []
    for word in element:
        if not bool(re.search(r'\d', word)):
            no_numbers.append(word)
    return no_numbers

def list_to_text(element):
    return ' '.join(element)

In [4]:
def preprocess_text_cols(dframes):
    for df in dframes:
        drop_keyword_and_location_cols(df)
        df['lower_text'] = df['text'].str.lower()
        df['text_no_punc'] = df['text'].apply(remove_punctuation_from_string)
        df['tokens_no_punc'] = df['text_no_punc'].apply(tokenize_words)
        df['tokens_no_punc_no_stopwords'] = df['tokens_no_punc'].apply(remove_stopwords)
        df['tokens_no_punc_no_stopwords_no_links'] = df['text_no_punc'].apply(remove_links_and_stopwords)
        df['lemmed_words'] = df['tokens_no_punc_no_stopwords_no_links'].apply(lemmatizing)
        df['lemmed_words_no_numbers'] = df['lemmed_words'].apply(remove_numbers)
        df['final_text'] = df['lemmed_words_no_numbers'].apply(list_to_text)
        
def create_new_features(dframes):
    for df in dframes:
        df['avg_word_length_feat'] = df['tokens_no_punc'].apply(avg_word_length)
        df['num_words_feat'] = df['tokens_no_punc'].apply(len)
        df['mentions'] = df['lower_text'].apply(extract_mentions)
        df['mention_count_feat'] = df['mentions'].apply(len)
        df['includes_mention_feat'] = df['mention_count_feat'].apply(includes_mention)
        df['stop_words'] = df['tokens_no_punc'].apply(extract_stopwords)
        df['stop_words_count_feat'] = df['stop_words'].apply(len)
        df['number_count_feat'] = df['text_no_punc'].apply(count_numbers)
        df['hashtags'] = df['lower_text'].apply(extract_hashtags)
        df['hashtag_count_feat'] = df['hashtags'].apply(len)
        df['includes_hashtag_feat'] = df['hashtag_count_feat'].apply(includes_hashtag)
        df['punctuation_count_feat'] = df['lower_text'].apply(punctuation_count)
        df['weblinks'] = df['lower_text'].apply(extract_weblinks)
        df['weblinks_count_feat'] = df['weblinks'].apply(len)
        df['includes_weblink_feat'] = df['weblinks_count_feat'].apply(includes_weblink)

In [5]:
preprocess_text_cols(dframes)

In [6]:
create_new_features(dframes)

In [7]:
def create_corpus(df):
    corpus = {}
    for index, row in df.iterrows():
        for word in row['lemmed_words_no_numbers']:
            if word in corpus:
                corpus[word] += 1
            else:
                corpus[word] = 1
    return corpus

word_dict = create_corpus(train)
sorted_word_dict = {k: v for k, v in sorted(word_dict.items(), key=lambda item: item[1], reverse=True)}

keys_to_remove = ['\x89', 'û', 'ûªs', '÷', 'ûª', '\x9d', 'ûò', 'ûªt', 'ûó' ]
for key in keys_to_remove:
    sorted_word_dict.pop(key)

corpus_dict = {k:v for (k,v) in sorted_word_dict.items() if v > 19}
corpus = [k for k,v in corpus_dict.items()]

In [8]:
tfidf_vect = TfidfVectorizer(vocabulary=corpus)
tfidf_vect_fit = tfidf_vect.fit(train['final_text'])
tfidf_train = tfidf_vect_fit.transform(train['final_text'])
tfidf_test = tfidf_vect_fit.transform(test['final_text'])
word_features_train_df = pd.DataFrame(tfidf_train.toarray())
word_features_test_df = pd.DataFrame(tfidf_test.toarray())

In [9]:
features = [col for col in train.columns if col.endswith('feat')]

In [10]:
train = pd.concat([train[features].reset_index(drop=True), word_features_train_df, target.reset_index(drop=True)], axis=1)
test = pd.concat([test[features].reset_index(drop=True), word_features_test_df], axis=1)

In [11]:
features = [col for col in train.columns if col not in ['target']]

In [12]:
train.head()

,avg_word_length_feat,num_words_feat,mention_count_feat,includes_mention_feat,stop_words_count_feat,number_count_feat,hashtag_count_feat,includes_hashtag_feat,punctuation_count_feat,weblinks_count_feat,includes_weblink_feat,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557,558,559,560,561,562,563,564,565,566,567,568,569,570,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,606,607,608,609,610,611,612,613,614,615,616,617,618,619,620,621,622,623,624,625,626,627,628,629,630,631,632,633,634,635,636,637,638,639,640,641,642,643,644,645,646,647,648,649,650,651,652,653,654,655,656,657,658,659,660,661,662,663,664,665,666,667,668,669,670,671,672,673,674,675,676,677,678,679,680,681,682,683,684,685,686,687,688,689,690,691,692,693,694,695,696,697,698,699,700,701,702,703,704,705,706,707,708,709,710,711,712,713,714,715,716,717,718,719,720,721,722,723,724,725,726,727,728,729,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,target
0,4.307692,13,0,0,6,0,1,1,1,0,0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.524997,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.582056,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

## Random Forest Classifier

In [13]:
start = time.time()

rf = RandomForestClassifier()
params = {'n_estimators':[10, 150, 300],
          'max_depth':[30, 60, 90, None]}
gs = GridSearchCV(rf, params, scoring='accuracy', cv=5, verbose=1)
gs_fit = gs.fit(train[features], target)
end = time.time()
print('Training Time (Minutes):', (end-start)/60)
results = pd.DataFrame(gs_fit.cv_results_).sort_values(by='mean_test_score', ascending=False)
display(results.head())

Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:  2.6min finished


Training Time (Minutes): 2.737782879670461


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
4,8.757596,0.294214,0.156338,0.020840,60,150,"{'max_depth': 60, 'n_estimators': 150}",0.699934,0.661852,0.676953,0.677398,0.734560,0.690139,0.025320,1
2,10.289869,0.155839,0.173789,0.007800,30,300,"{'max_depth': 30, 'n_estimators': 300}",0.697965,0.659225,0.688772,0.663601,0.741130,0.690139,0.029405,2
1,7.299641,0.792145,0.137704,0.021195,30,150,"{'max_depth': 30, 'n_estimators': 150}",0.693368,0.666448,0.686802,0.662286,0.732589,0.688299,0.025074,3
5,18.212009,0.419138,0.268881,0.010724,60,300,"{'max_depth': 60, 'n_estimators': 300}",0.692712,0.660538,0.675640,0.672799,0.733903,0.687118,0.025547,4
11,23.024929,5.642345,0.290902,0.074446,None,300,"{'max_depth': None, 'n_estimators': 300}",0.704531,0.659882,0.666448,0.659658,0.737845,0.685673,0.030949,5


In [14]:
#start = time.time()

#gbc = GradientBoostingClassifier()
#params = {'loss': ['deviance', 'exponential'],
#          'learning_rate': [0.1, 0.05, 0.2],
#          'n_estimators': [100, 10, 200]}
#gs = GridSearchCV(gbc, params, scoring='accuracy', cv=5, verbose=1)
#gs_fit = gs.fit(train[features], target)
#end = time.time()
#print('Training Time (Minutes):', (end-start)/60)
#results = pd.DataFrame(gs_fit.cv_results_).sort_values(by='mean_test_score', ascending=False)
#display(results)

In [15]:
winner = gs.best_estimator_
y_pred = winner.predict(test[features])

In [16]:
submission = pd.DataFrame({'id': test_ids, 'target': y_pred})

In [17]:
submission.to_csv('submission.csv', index=False)